## To do

- [ ] Allow subsets (Sensitivity Analysis: samplingMethodData.method)

In [367]:
import json, importlib, pprint 
pp = pprint.PrettyPrinter(indent=4)
from copy import deepcopy
import carsonpy as csn

In [368]:
importlib.reload(csn)

<module 'carsonpy' from '/mnt/c/Users/claud/depot/cmp/jardin_py/carsonpy.py'>

In [369]:
with open('uq.schema.json') as f: schema = json.load(f)

In [370]:
# suite = csn.schemaCartesianProduct(schema['properties']['calibrationMethodData'])
# suite = csn.schemaCartesianProduct(schema)

In [593]:
# def product(*args):
#     res = []
#     product_helper([], args, 0, res)
#     return res

# def product_helper(prefix, pools, offset, res):
#     if offset >= len(pools):
#         res.append(prefix)
#         return

#     pool = pools[offset]

#     for i in range(len(pool)):
#         product_helper(prefix + [pool[i]], pools, offset + 1, res)

def product(*seqs):
    if not seqs:
        return [[]]
    else:
        for x in seqs[0]:
            result = [[]]
            for p in product(*seqs[1:]):
                
        return [[x] + p for x in seqs[0] for p in product(*seqs[1:])]
    
def product(pool):
    if not pool:
        return [[]]
    else:
        for x in pool[0]:
            for p in product(pool[1:]):
                
        return [[x] + p for x in seqs[0] for p in product(*seqs[1:])]

In [594]:
options = {
    "number": [1,2,3], 
    "color": ["orange","blue"] 
}

In [595]:
list(options.values())

[[1, 2, 3], ['orange', 'blue']]

In [596]:
product(list(options.values()))

seqs:  [[1, 2, 3], ['orange', 'blue']]
seqs:  ['orange', 'blue']
seqs:  blue
seqs:  l


IndexError: string index out of range

In [564]:
def objProduct(obj: dict, parent={}) -> list:
    prod = product(list(obj.values()))
    keys = obj.keys()
    out = []
    for values in prod:
        var = parent.copy()
        for i,key in enumerate(keys):
            var.update({key: values[i]})
        out.append(var)
    return out

In [560]:
objProduct(options)

TypeError: 'int' object is not subscriptable

In [ ]:
def parsePropVariations(schema: dict, parent: dict = {}, parent_key: str = '') -> dict:
    variations: list = []
    properties: dict = schema['properties']
    
    if 'oneOf' in schema:
        schema_variations: list = schema['oneOf']
        for schema_variation in schema['oneOf']: #call-01 = row-02
            variation_parent: dict = deepcopy(parent)
            
            id_props: dict = schema_variation['properties']
            dp_props: list = schema_variation['required']
            
            for prop_key,prop_val in id_props.items():
                print(id_props[prop_key]['const'])
                key: str = parent_key + '.' + prop_key if parent_key else prop_key
                variation_parent.update({key: prop_val['const']})
            
            prop_schemas = []
            enum_props = {}
            
            for prop_name in dp_props:
                # extract constant fields and identify variation generators
                prop_key:str = prop_name #dp_props[k]
                key: str = parent_key + '.' + prop_key if parent_key else prop_key
                prop = properties[prop_key]
                print('key: {}'.format(prop_key))
                if prop['type'] == 'object':
                    prop_schemas.append(prop_key)
                elif 'enum' in prop:
                    enum_props.update({key: prop['enum']})
                elif 'default' in prop:
                    variation_parent.update({key: prop['default']})
                else:
#                     pass
                    raise Exception("Ambiguous schema.")
    
            # objects cannot have both 'enum' properties and 'oneOf' properties
            assert (not prop_schemas) or (not enum_props)
            
            if prop_schemas:
                for prop_key in prop_schemas:
                    prop_schema_variations: list(dict) = parsePropVariations(properties[prop_key], variation_parent, prop_key)
                    variations.extend(prop_schema_variations)
                    
            elif enum_props:
                enum_variations: list(dict) = objProduct(enum_props, variation_parent)
                num_variations = len(enum_variations)
                variations.extend(enum_variations)
            else:
#                 print()
                variations.append(variation_parent)
    else:
        variation_parent: dict = deepcopy(parent)
        enum_props = {}
        for prop_key in properties:
            key: str = parent_key + '.' + prop_key if parent_key else prop_key
            if 'enum' in properties[prop_key]:
                enum_props.update({key: properties[prop_key]['enum']})
                
            elif properties[prop_key]['type'] != 'object' and 'default' in properties[prop_key]: 
                variation_parent.update({key: properties[prop_key]['default']})
            
        enum_variations: list(dict) = objProduct(enum_props, variation_parent)
        num_variations = len(enum_variations)
        variations.extend(enum_variations)    
    return variations

In [ ]:
%%capture
output = parsePropVariations(schema)

In [ ]:
assert (109==len(output))

In [ ]:
output

In [ ]:

{i['uqType']:len([j for j in output if j['uqType']==i['uqType']]) for i in output}

In [ ]:
[d for d in output if 'Parameter Estimation' in d['uqType']]


In [ ]:
# %%capture
# smd = parsePropVariations(schema['properties']['samplingMethodData'])

In [ ]:
len([d for d in smd if 'Polynomial' in d['method']])
# smd
# smd

In [ ]:
# %%capture
rmd = parsePropVariations(schema['properties']['reliabilityMethodData'])
len([d for d in rmd if 'Global' in d['method']])

In [ ]:
'Gaussian Process Regression'